In [ ]:
import docx2txt
import json
import numpy as np
import openai
import requests

In [8]:
# read the docx file
text = docx2txt.process('story.docx')

In [ ]:
# clean the data
min_len = 50
filter_story = [line for line in text.split('\n') if len(line) > min_len]

In [ ]:
def get_lan(comment, lan="en"):
    # Using the api call to translate the comment to any language in the azure translator
    subscription_key = ""
    subscription_region = ""
    domain = ""
    # Set the headers for the request
    headers = {
        "Ocp-Apim-Subscription-Key": subscription_key,
        "Ocp-Apim-Subscription-Region": subscription_region,
        "Content-Type": "application/json",
    }

    url = f"https://{domain}.cognitiveservices.azure.com/translator/text/v3.0/translate?to={lan}"

    # Set the JSON data to send in the request body
    data = json.dumps([{"Text": comment}])

    # Send the POST request
    response_tanslate = requests.post(url, headers=headers, data=data)
    return json.loads(response_tanslate.text)[0]["translations"][0]["text"]

In [ ]:
# create the dictionary and translate the text
data_story = {i:{'text_he':line, 'text_en':get_lan(line, lan='en')} for i, line in enumerate(filter_story)}

# write a json file
with open('data_story.json', 'w') as outfile:
    json.dump(data_story, outfile)


In [11]:
# read json data tree.json
with open('tree.json') as json_file:
    data_story = json.load(json_file)

In [15]:
# ctrate a dictionary with embeded numpy array of the text
dic_np = {key:openai.Embedding.create(input=data_story[key]['text_en'],      
 						 engine="text-embedding-ada-002"
                                	)["data"][0]["embedding"]
          for key in data_story}

# write the dictonery with numpy array to a .npz file
np.savez_compressed('dic_np.npz', **dic_np)

# read the .npz file
loaded_dict = np.load('dic_np.npz', allow_pickle=True)

# creating the matrix embedding
matrix_embedding = np.array([loaded_dict[key] for key in loaded_dict])


Search the text

In [ ]:
# enter text here
enter_txt_he = ''

In [ ]:
# get the matrix embedding
matrix_embedding = np.array([loaded_dict[key] for key in loaded_dict])
# get the question in english
enter_txt_en = get_lan(enter_txt_he, lan="en")

# get the question embedding
embedding = openai.Embedding.create(input=enter_txt_en, engine="text-embedding-ada-002")["data"][0]["embedding"]

# calculate the cosine similarity
cosine_vector = np.dot(matrix_embedding, embedding) / (
    np.linalg.norm(matrix_embedding) * np.linalg.norm(embedding))

# get the index of the most similar quote
idx_answer = np.argmax(cosine_vector)

# get the answer in the original language (hebrew)
answer = data_story[idx_answer]["text_he"]
